
# Tarea 11: Red Neuronal para el California Housing Dataset

En esta tarea vamos a proponer una alternativa basada en redes neuronales al modelo Random Forest previamente entrenado, que obtuvo un RMSE de aproximadamente **41,600 $**. Mantendremos el mismo pipeline de preprocesamiento para poder comparar de forma justa ambos modelos.


In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from preprocessing import cat_pipeline  # También podríamos importar un ColumnTransformer completo


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

housing = pd.read_csv("housing.csv")
housing.head()


In [ ]:

# Asumimos que el preprocesador completo está implementado como 'full_pipeline' en preprocessing.py
from preprocessing import full_pipeline

X = housing.drop("median_house_value", axis=1)
y = housing["median_house_value"]

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Aplicamos el pipeline
X_train_prep = full_pipeline.fit_transform(X_train)
X_valid_prep = full_pipeline.transform(X_valid)
X_test_prep = full_pipeline.transform(X_test)


In [ ]:

model = keras.Sequential([
    layers.Input(shape=X_train_prep.shape[1]),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')


In [ ]:

early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(
    X_train_prep, y_train,
    validation_data=(X_valid_prep, y_valid),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping]
)


In [ ]:

y_pred = model.predict(X_test_prep)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE en test: {rmse:.2f} $")



## Conclusiones

- Se ha entrenado una red neuronal simple con dos capas ocultas para el problema de regresión.
- Se ha mantenido el mismo pipeline de preprocesamiento que en el modelo de Random Forest para permitir la comparación justa.
- El RMSE obtenido se compara con el baseline de ~41,600 $.
